<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/HOML_Exercise_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#MNIST data set
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#1
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
#param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5]}]
param_grid = [{'weights': ["distance"], 'n_neighbors': [4]}]
knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=2, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] n_neighbors=4, weights=distance .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................. n_neighbors=4, weights=distance, total=18.1min
[CV] n_neighbors=4, weights=distance .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 18.1min remaining:    0.0s


[CV] .................. n_neighbors=4, weights=distance, total=18.1min


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 36.1min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [4], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [ ]:
from sklearn.metrics import accuracy_score
print(grid_search.best_params_)
print(accuracy_score(y_test, grid_search.predict(X_test)))

{'n_neighbors': 4, 'weights': 'distance'}
0.9731428571428572


In [ ]:
#2
import numpy as np
from scipy.ndimage.interpolation import shift

#이미지 이동
def shift_image(image, dx=0, dy=0, shift1=False):
    image = image.reshape((28, 28))
    if shift1:
        shifted_image = []
        for dx, dy in ((1,0), (0,1), (-1,0), (0,-1)):
            shifted_image.append(shift(image, [dy, dx], cval=0, mode='constant').reshape([-1]))
        return shifted_image

    shifted_image_down = shift(image, [dy, dx], cval=0, mode='constant')
    return shifted_image_down.reshape([-1])

#데이터 복사
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

#이동 이미지 데이터 추가
for image, label in zip(X_train, y_train):
    for shifted_image in shift_image(image, shift1=True):
        X_train_augmented.append(shifted_image)
        y_train_augmented.append(label)

#numpy 배열 변환
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

#셔플
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

In [ ]:

knn_clf = KNeighborsClassifier(**grid_search.best_params_)
knn_clf.fit(X_train_augmented, y_train_augmented)
print(accuracy_score(y_test, knn_clf.predict(X_test)))